In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import xarray as xr

import oggm
import os
from oggm import cfg, utils, workflow, tasks, graphics
from oggm.core import inversion
cfg.initialize(logging_level='WORKFLOW')
products_dir = '/home/pelto/Desktop/ice_thick/'

2019-10-04 13:25:58: oggm.cfg: Using configuration file: /home/pelto/anaconda2/envs/oggm_env/lib/python3.6/site-packages/oggm/params.cfg


# Nordic

In [ ]:
utils.get_rgi_dir(version='61')

sh = utils.get_rgi_region_file('02', version='61')

gdf = gpd.read_file(sh)
# gdf.head()

# OGGM set-up
cfg.PATHS['working_dir'] = utils.gettempdir(dirname='nordic_intersects', reset=True)
cfg.PARAMS['border'] = 10

# Get the geometry and plot it
gl = utils.get_rgi_glacier_entities(['RGI60-02.04264'], version='61') ##unless I put the version here it doesn't call V61
# gl.plot(edgecolor='k');

# We simulate the case where we only have the geometry, nothing else
divides = gpd.read_file('/home/pelto/GIS/outlines/nordic_divided_2014.shp') #utils.get_demo_file('divides_alps.shp'))
divides = divides.loc[divides.RGIId == 'RGI60-02.04264'][['geometry']]
# divides

template = pd.concat([gl]*2, ignore_index=True)
# template

# Attributes
template['RGIId'] = ['RGI60-02.04264', 'RGI60-02.04264_d02']
template['Name'] = ['Nordic', 'Escargot']
# Geometries
template['geometry'] = divides['geometry'].values
# Center point
for i, geom in template[['geometry']].iterrows():
    cenlon, cenlat = geom.geometry.centroid.xy
    template.loc[i, 'CenLon'] = cenlon
    template.loc[i, 'CenLat'] = cenlat
# This is important to properly georeference the file
import salem
template.crs = salem.wgs84.srs

# Attributes
template['RGIId'] = ['RGI60-02.04264', 'RGI60-02.04264_d02']
template['Name'] = ['Nordic', 'Escargot']
# Geometries
template['geometry'] = divides['geometry'].values
# Center point
for i, geom in template[['geometry']].iterrows():
    cenlon, cenlat = geom.geometry.centroid.xy
    template.loc[i, 'CenLon'] = cenlon
    template.loc[i, 'CenLat'] = cenlat
# This is important to properly georeference the file
import salem
template.crs = salem.wgs84.srs

nord_new_shape_path = os.path.join(cfg.PATHS['working_dir'], 'nordic_divided.shp')
template.to_file(nord_new_shape_path)

intersects_wna = gpd.read_file(utils.get_rgi_intersects_region_file('02'))
intersects_nord = intersects_wna.loc[(intersects_wna.RGIId_1 == 'RGI60-02.04264') | (intersects_wna.RGIId_2 == 'RGI60-02.04264')]
# intersects_nord

from rgitools.funcs import compute_intersects
new_intersects = compute_intersects(template)  #for_intersects  ##removed as no adjacent glacier at Nordic

nord_intersects_path = os.path.join(cfg.PATHS['working_dir'], 'nordic_divided_intersects.shp')
new_intersects.to_file(nord_intersects_path)

# This is important! We tell OGGM to recompute the glacier area for us
cfg.PARAMS['use_rgi_area'] = False
# This is the default anyway, but we set it here to be sure
cfg.PARAMS['use_intersects'] = True

# This is important!
cfg.set_intersects_db(nord_intersects_path)
# nord_intersects_path

rgidf = gpd.read_file(nord_new_shape_path)
# rgidf

gdirs = workflow.init_glacier_regions(rgidf, reset=True, force=True)

# Illecillewaet

In [ ]:
sh = gpd.read_file('/home/pelto/GIS/outlines/illec_divided_2016.shp') #nordic_divided_2014.shp') #utils.get_demo_file('divides_alps.shp'))
sh

# OGGM set-up
cfg.PATHS['working_dir'] = utils.gettempdir(dirname='illec_intersects', reset=True) #'nordic_intersects'
cfg.PARAMS['border'] = 10

divides = gpd.read_file('/home/pelto/GIS/outlines/illec_divided_2016.shp')

illec_new_shape_path = os.path.join(cfg.PATHS['working_dir'], 'illec_divided.shp')
divides.to_file(illec_new_shape_path) #template for Nordic, divides for Illec

from rgitools.funcs import compute_intersects
new_intersects = compute_intersects(divides)  #for_intersects  ##removed as no adjacent glacier at Nordic

illec_intersects_path = os.path.join(cfg.PATHS['working_dir'], 'illec_divided_intersects.shp')
new_intersects.to_file(illec_intersects_path)

# This is important! We tell OGGM to recompute the glacier area for us
cfg.PARAMS['use_rgi_area'] = False
# This is the default anyway, but we set it here to be sure
cfg.PARAMS['use_intersects'] = True

# This is important!
cfg.set_intersects_db(illec_intersects_path)
illec_intersects_path

# This is important again - standard OGGM 
rgidf = gpd.read_file(illec_new_shape_path)
rgidf

gdirs = workflow.init_glacier_regions(rgidf, reset=True, force=True)